In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import Data

The following is a list of files and their description

+ sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
+ test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
+ sample_submission.csv - a sample submission file in the correct format.
+ items.csv - supplemental information about the items/products.
+ item_categories.csv  - supplemental information about the items categories.
+ shops.csv- supplemental information about the shops.

In [2]:
# Import data from 'all' folder
item_cats = pd.read_csv('../all/item_categories.csv')
items = pd.read_csv('../all/items.csv')
shops = pd.read_csv('../all/shops.csv')

sales_train = pd.read_csv('../all/sales_train.csv.gz', parse_dates = True, compression = 'gzip')

In [3]:
item_cats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [4]:
item_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
item_category_name    84 non-null object
item_category_id      84 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [5]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [7]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [8]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [9]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [10]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


The descriptions of each of the columns are listed below.

+ **ID** - an Id that represents a (Shop, Item) tuple within the test set
+ **shop_id** - unique identifier of a shop
+ **item_id** - unique identifier of a product
+ **item_category_id** - unique identifier of item category
+ **item_cnt_day** - number of products sold. *You are predicting a monthly amount of this measure*
+ **item_price** - current price of an item
+ **date** - date in format dd/mm/yyyy
+ **date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
+ **item_name** - name of item
+ **shop_name** - name of shop
+ **item_category_name** - name of item category

## Test Set
In order to create predictions for the test set, the test set needs to be explored to see what information is provided. It's stated that these numbers are for November 2015 which is the next month succeeding the training set. Once we know what is given in the test set, we can figure out what is needed in the training set.

In [11]:
# Import the test set
sales_test = pd.read_csv('../all/test.csv.gz', compression = 'gzip')

In [12]:
sales_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [13]:
sales_test.describe()

,ID,shop_id,item_id
count,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627
std,61834.358168,17.561933,6252.644590
min,0.000000,2.000000,30.000000
25%,53549.750000,16.000000,5381.500000
50%,107099.500000,34.500000,11203.000000
75%,160649.250000,47.000000,16071.500000
max,214199.000000,59.000000,22167.000000


In [14]:
sales_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


It seems all that provided for the test set is the ``shop_id`` and ``item_id`` so adding more columns might be necessary. Those columns could be:
+ ``date`` or ``date_block_num``
+ ``item_category_id``

# Clean Data

The first thing I notice here is that the item and shop names aren't in English but that shouldn't be a problem. Second, the ``date`` column in **sales_train** is in an awkward format which needs to be handled immediately. With that in mind, the column ``date_block_num`` may be deemed unnecessary since it's correlated to the actual ``date`` column. When that is resolved, it may be useful to combine the ``item_category_id`` column into the **sales_train**, and _test.csv_ for that matter, data sets which could be useful when exploring the data. One more thing that comes to mind is to convert the ID columns to categorical variables since there shouldn't be any fractional ID numbers and no relation between shop/item IDs.

In [15]:
# Format 'date' column
sales_train['date'] = pd.to_datetime(sales_train['date'], format = '%d.%m.%Y')

In [16]:
# Merge items and item_cats
# This is to be used later
item_id_cat_merged = pd.merge(items, item_cats, how='left', on='item_category_id')

item_id_cat_merged.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD


The objective of this problem is to forecast the total amount of products sold in every shop, specifically for the provided test set. It's to be noted that the list of shops and products change from month to month which needs to be accounted for. The problem statement in itself is a little ambitguous, but it's stated that we want to predict ``item_cnt_day``. A future project could investigate how much money shops make, which item categories do well, and which items tend to sell more.

# Resample Data

Now that the training and test sets are clean, and there doesn't seem to be any missing values, the next step is to get the data in a format necessary for this problem. The problem wants to look at monthly sales by shop. That means the dates need to be resampled by month and grouped by ``shop_id`` and ``item_id`` in the training set and dates need to be added to the test set.

In [17]:
# Add column for profits
sales_train['profits'] = sales_train['item_price'] * sales_train['item_cnt_day']

In [18]:
# Resample by month and group by date, shop_id, and item_id. Take sum to get total item_cnt_day
sales_train_grouped = sales_train.groupby([pd.Grouper(key='date', freq='m'), 'date_block_num', 'shop_id', 'item_id']).sum()

In [19]:
# Reset index so a merge on item_id is possible
sales_train_grouped.reset_index(inplace = True)

In [20]:
# Merge sales_train_grouped and item_id_cat_merged
sales_train_merge = pd.merge(sales_train_grouped, item_id_cat_merged[['item_id', 'item_category_id']], how='left', on='item_id')

In [21]:
# Convert necessary columns to categories in training set
sales_train_merge['shop_id'] = sales_train_merge['shop_id'].astype('category')
sales_train_merge['item_id'] = sales_train_merge['item_id'].astype('category')
sales_train_merge['date_block_num'] = sales_train_merge['date_block_num'].astype('category')
sales_train_merge['item_category_id'] = sales_train_merge['item_category_id'].astype('category')

In [22]:
sales_train_merge.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,profits,item_category_id
0,2013-01-31,0,0,32,884.0,6.0,1326.0,40
1,2013-01-31,0,0,33,1041.0,3.0,1041.0,37
2,2013-01-31,0,0,35,247.0,1.0,247.0,40
3,2013-01-31,0,0,43,221.0,1.0,221.0,40
4,2013-01-31,0,0,51,257.0,2.0,257.0,57
5,2013-01-31,0,0,61,195.0,1.0,195.0,43
6,2013-01-31,0,0,75,76.0,1.0,76.0,40
7,2013-01-31,0,0,88,76.0,1.0,76.0,40
8,2013-01-31,0,0,95,193.0,1.0,193.0,40
9,2013-01-31,0,0,96,70.0,1.0,70.0,40


In [23]:
sales_train_merge.describe()

,item_price,item_cnt_day,profits
count,1.609124e+06,1.609124e+06,1.609124e+06
mean,1.625363e+03,2.267200e+00,2.112284e+03
std,5.701611e+03,8.649882e+00,1.531646e+04
min,9.000000e-02,-2.200000e+01,-3.378700e+04
25%,2.490000e+02,1.000000e+00,2.490000e+02
50%,4.990000e+02,1.000000e+00,5.370000e+02
75%,1.398000e+03,2.000000e+00,1.490000e+03
max,6.719300e+05,2.253000e+03,6.264230e+06


In [24]:
sales_train_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609124 entries, 0 to 1609123
Data columns (total 8 columns):
date                1609124 non-null datetime64[ns]
date_block_num      1609124 non-null category
shop_id             1609124 non-null category
item_id             1609124 non-null category
item_price          1609124 non-null float64
item_cnt_day        1609124 non-null float64
profits             1609124 non-null float64
item_category_id    1609124 non-null category
dtypes: category(4), datetime64[ns](1), float64(3)
memory usage: 69.9 MB


In [25]:
# Add columns to sales_test
sales_test_merge = pd.merge(sales_test, item_id_cat_merged[['item_id', 'item_category_id']], how = 'left', on='item_id')

In [26]:
# Add date column(s)
sales_test_merge['date'] = pd.to_datetime('30-11-2015', format = '%d-%m-%Y')

In [27]:
# Add date_block_num
sales_test_merge['date_block_num'] = 34

In [28]:
# Convert necessary columns into categories in test set
sales_test_merge['shop_id'] = sales_test_merge['shop_id'].astype('category')
sales_test_merge['item_id'] = sales_test_merge['item_id'].astype('category')
sales_test_merge['date_block_num'] = sales_test_merge['date_block_num'].astype('category')
sales_test_merge['item_category_id'] = sales_test_merge['item_category_id'].astype('category')

In [29]:
sales_test_merge.head(10)

,ID,shop_id,item_id,item_category_id,date,date_block_num
0,0,5,5037,19,2015-11-30,34
1,1,5,5320,55,2015-11-30,34
2,2,5,5233,19,2015-11-30,34
3,3,5,5232,23,2015-11-30,34
4,4,5,5268,20,2015-11-30,34
5,5,5,5039,23,2015-11-30,34
6,6,5,5041,20,2015-11-30,34
7,7,5,5046,55,2015-11-30,34
8,8,5,5319,55,2015-11-30,34
9,9,5,5003,20,2015-11-30,34


In [30]:
sales_test_merge.describe()

,ID
count,214200.000000
mean,107099.500000
std,61834.358168
min,0.000000
25%,53549.750000
50%,107099.500000
75%,160649.250000
max,214199.000000


In [31]:
sales_test_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 6 columns):
ID                  214200 non-null int64
shop_id             214200 non-null category
item_id             214200 non-null category
item_category_id    214200 non-null category
date                214200 non-null datetime64[ns]
date_block_num      214200 non-null category
dtypes: category(4), datetime64[ns](1), int64(1)
memory usage: 6.1 MB


In [32]:
# Export merged data sets
sales_train_merge.to_csv('../Data/sales_train_merge.csv')
sales_test_merge.to_csv('../Data/sales_test_merge.csv')